# Contents and why we need this lab

This lab is about implementing neural networks yourself in NumPy before we start using other frameworks which hide some of the computation from you. It builds on the first lab where you derived the equations for neural network forward and backward propagation and gradient descent parameter updates.

# External sources of information

1. Jupyter notebook. You can find more information about Jupyter notebooks [here](https://jupyter.org/). It will come as part of the [Anaconda](https://www.anaconda.com/) Python installation. 
2. [NumPy](https://numpy.org/). Part of Anaconda distribution. If you already know how to program most things about Python and NumPy can be found through Google search. 


# This notebook will follow the next steps:

1. Data generation
2. Initialization of parameters
3. Definition of activation functions   
4. A short explanation of numpy's einsum function
5. Forward pass
6. Backward pass (backward pass and finite differences)
7. Training loop 
8. Testing your model
9. Further extensions

# Create an artificial dataset to play with

We create a non-linear 1d regression task. The generator supports various noise levels and it creates train, validation and test sets. You can modify it yourself if you want more or less challenging tasks.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

In [ ]:
def data_generator(noise=0.1, n_samples=300, D1=True):
    # Create covariates and response variable
    if D1:
        X = np.linspace(-3, 3, num=n_samples).reshape(-1,1) # 1-D
        np.random.shuffle(X)
        y = np.random.normal((0.5*np.sin(X[:,0]*3) + X[:,0]), noise) # 1-D with trend
    else:
        X = np.random.multivariate_normal(np.zeros(3), noise*np.eye(3), size = n_samples) # 3-D
        np.random.shuffle(X)    
        y = np.sin(X[:,0]) - 5*(X[:,1]**2) + 0.5*X[:,2] # 3-D

    # Stack them together vertically to split data set
    data_set = np.vstack((X.T,y)).T
    
    train, validation, test = np.split(data_set, [int(0.35*n_samples), int(0.7*n_samples)], axis=0)
    
    # Standardization of the data, remember we do the standardization with the training set mean and standard deviation
    train_mu = np.mean(train, axis=0)
    train_sigma = np.std(train, axis=0)
    
    train = (train-train_mu)/train_sigma
    validation = (validation-train_mu)/train_sigma
    test = (test-train_mu)/train_sigma
    
    x_train, x_validation, x_test = train[:,:-1], validation[:,:-1], test[:,:-1]
    y_train, y_validation, y_test = train[:,-1], validation[:,-1], test[:,-1]

    return x_train, y_train,  x_validation, y_validation, x_test, y_test

In [ ]:
D1 = True
x_train, y_train,  x_validation, y_validation, x_test, y_test = data_generator(noise=0.5, D1=D1)

In [ ]:
if D1:
    plt.scatter(x_train[:,0], y_train);
    plt.scatter(x_validation[:,0], y_validation);
    plt.scatter(x_test[:,0], y_test);
else:
    plt.scatter(x_train[:,1], y_train);
    plt.scatter(x_validation[:,1], y_validation);
    plt.scatter(x_test[:,1], y_test);
plt.show()

# Initialization

The steps to create a feed forward neural network are the following:

1. **Number of hidden layer and hidden units**. We have to define the number of hidden units in each layer. We are going to save these numbers in a list "L" that is going to start with our input dimensionality (the number of features in X) and is going to finish with our output dimensionality (the size of Y). Anything in between these values are going to be hidden layers and the number of hidden units in each hidden layer is defined by the researcher. Remember that for each unit in each layer (besides the first one, according to our list L) there is a bias term.
2. **Activation functions** for each hidden layer. Each hidden layer in your list must have an activation function (it can also be the linear activation which is equivalent to identity function). The power of neural networks comes from non-linear activation functions that learn representations (features) from the data allowing us to learn from it. 
3. **Parameter initialization**. We will initialize the weights to have random values. This is done in practice by drawing pseudo random numbers from a Gaussian or uniform distribution. It turns out that for deeper models we have to be careful about how we scale the random numbers. This will be the topic of the exercise below. For now we will just use unit variance Gaussians.  

Our initialization will work as follows: 

For each layer of the neural network defined in L, initialize a matrix of weights of size (units_in, units_out) from a random normal distribution [np.random.normal()](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.normal.html) and save them in a list called "layers". For each layer in our neural network, initialize a matrix of weights of size (1, units_out) as above and save them in a list called "bias". The function should return a tuple (layers, bias). The length of our lists must be len(L)-1.

In [ ]:
# Initialize neural network:
# the NN is a tuple with a list with weights and list with biases
def init_NN(L):
    """
    Function that initializes our feed-forward neural network. 
    Input: 
    L: list of integers. The first element must be equal to the number of features of x and the last element 
        must be the number of outputs in the network.
    Output:
    A tuple of:
    weights: a list with randomly initialized weights of shape (in units, out units) each. The units are the ones we defined in L.
        For example, if L = [2, 3, 4] layers must be a list with a first element of shape (2, 3) and a second elemtn of shape (3, 4). 
        The length of layers must be len(L)-1
    biases: a list with randomly initialized biases of shape (1, out_units) each. For the example above, bias would be a list of length
        2 with a first element of shape (1, 3) and a second element of shape (1, 4).
    """
    weights = []
    biases  = []
    for i in range(len(L)-1):
        weights.append(np.random.normal(loc=0.0, scale=1.0, size=[L[i],L[i+1]])) 
        biases.append(np.random.normal(loc=0.0, scale=1.0, size=[1, L[i+1]]))     
        
    return (weights, biases)

# Initialize the unit test neural network:
# Same steps as above but we will not initialize the weights randomly.
def init_NN_UT(L):
    weights = []
    biases  = []
    for i in range(len(L)-1):
        weights.append(np.ones((L[i],L[i+1]))) 
        biases.append(np.ones((1, L[i+1])))     
        
    return (weights, biases)

# Initializer the unit test neural network
L_UT  = [3, 5, 1]
NN_UT = init_NN_UT(L_UT)

## Exercise a) Print all network parameters

Make a function that prints all parameters (weights and biases) with information about in which layer the parameters are.

In [ ]:
# Insert code here

# Advanced initialization schemes

If we are not careful with initialization, the signals we propagate forward ($a^{(l)}$, $l=1,\ldots,L$) and backward ($\delta^l$, $l=L,L-1,\ldots,1$) can blow up or shrink to zero. A statistical analysis of the variance of the signals for different activation functions can be found in these two papers: [Glorot initialization](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf) and [He initialization](https://arxiv.org/pdf/1502.01852v1.pdf). 

The result of the analyses are proposals for how to make the initialization such that the variance of the signals (forward and backward) are kept approxmimatly constant when propagating from layer to layer. The exact expressions depend upon the non-linear activation function used. In Glorot initialization, the aim is to keep both the forward and backward variances constant whereas He only aims at keeping the variance in the forward pass constant.

We define $n_{in}$ and $n_{out}$ as the number of input units and output units of a particular layer. 

The Glorot initialization has the form: 

$$w_{ij} \sim N \bigg( 0, \, \frac{2 \alpha }{n_{in} + n_{out}} \bigg) \ . $$

where $N(\mu,\sigma^2)$ is a Gaussian distribution with mean $\mu$ and variance $\sigma^2$ and $\alpha$ is a parameter that depends upon the activation function used. For $\tanh$, $\alpha=1$ and for Rectified Linear Unit (ReLU) activations, $\alpha=2$. (It is also possible to use a uniform distribution for initialization, see [this blog post](https://mmuratarat.github.io/2019-02-25/xavier-glorot-he-weight-init).) 

The He initialization is very similar

$$w_{ij} \sim N \bigg( 0, \, \frac{\alpha}{n_{in}} \bigg) \ . $$

## Exercise b) Glorot and He initialization

Implement these initialization schemes by modifying the code given below.

**NOTE:** The Gaussian is defined as $N( \mu, \, \sigma^{2})$ but Numpy takes $\sigma$ as argument.

Explain briefly how you would test numerically that these initializations have the sought after property. Hint: See plots in Glorot paper.

In [ ]:
## Glorot
def init_NN_glorot_Tanh(L):
    """
    Initializer using the glorot initialization scheme
    """
    weights = []
    biases  = []
    for i in range(len(L)-1):
        std = 1.0 # <- replace with proper initialization
        weights.append(np.random.normal(loc=0.0, scale=std, size=[L[i],L[i+1]])) 
        biases.append(np.random.normal(loc=0.0, scale=std, size=[1, L[i+1]]))       
        
    return (weights, biases)

## He
def init_NN_he_ReLU(L):
    """
    Initializer using the He initialization scheme
    """
    weights = []
    biases  = []
    for i in range(len(L)-1):
        std = 1.0 # <- replace with proper initialization
        weights.append(np.random.normal(loc=0.0, scale=std, size=[L[i],L[i+1]])) 
        biases.append(np.random.normal(loc=0.0, scale=std, size=[1, L[i+1]]))       
        
    return (weights, biases)

# Activation functions

To have a full definition of the neural network, we must define an activation function for every layer in our list L (again, exluding the first term, which is the number of input dimensions). Several activation functions have been proposed and have different characteristics. Here, we will implement the linear activation function (the identity function), the sigmoid activation function (squeeshes the outcome of each neuron into the $[0, 1]$ range), the Hyperbolic Tangent (Tanh) that squeeshes the outcome of each neuron to $[-1, 1]$ and the Rectified Linear Unit (ReLU). 

We will also include the derivative in the function. We need this in order to do our back-propagation algorithm. Don't rush, we will get there soon. For any neural network, save the activation functions in a list. This list must be of size len(L)-1.

## Linear activation

In [ ]:
def Linear(x, derivative=False):
    """
    Computes the element-wise Linear activation function for an array x
    inputs:
    x: The array where the function is applied
    derivative: if set to True will return the derivative instead of the forward pass
    """
    
    if derivative:              # Return the derivative of the function evaluated at x
        return np.ones_like(x)
    else:                       # Return the forward pass of the function at x
        return x

## Sigmoid activation

In [ ]:
def Sigmoid(x, derivative=False):
    """
    Computes the element-wise Sigmoid activation function for an array x
    inputs:
    x: The array where the function is applied
    derivative: if set to True will return the derivative instead of the forward pass
    """
    f = 1/(1+np.exp(-x))
    
    if derivative:              # Return the derivative of the function evaluated at x
        return f*(1-f)
    else:                       # Return the forward pass of the function at x
        return f

## Hyperbolic Tangent activation

In [ ]:
def Tanh(x, derivative=False):
    """
    Computes the element-wise Sigmoid activation function for an array x
    inputs:
    x: The array where the function is applied
    derivative: if set to True will return the derivative instead of the forward pass
    """
    f = (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
    
    if derivative:              # Return the derivative of the function evaluated at x
        return 1-f**2
    else:                       # Return the forward pass of the function at x
        return f

## Rectifier linear unit (ReLU)

In [ ]:
def ReLU(x, derivative=False):
    """
    Computes the element-wise Rectifier Linear Unit activation function for an array x
    inputs:
    x: The array where the function is applied
    derivative: if set to True will return the derivative instead of the forward pass
    """
    
    if derivative:              # Return the derivative of the function evaluated at x
        return (x>0).astype(int)
    else:                       # Return the forward pass of the function at x
        return np.maximum(x, 0)

## Visualization

Now that we have defined our activation functions we can visualize them to see what they look like:

In [ ]:
x = np.linspace(-6, 6, 100)
units = {
    "Linear": lambda x: Linear(x),
    "Sigmoid": lambda x: Sigmoid(x),
    "ReLU": lambda x: ReLU(x),
    "tanh": lambda x: Tanh(x)
}

plt.figure(figsize=(5, 5))
[plt.plot(x, unit(x), label=unit_name, lw=2) for unit_name, unit in units.items()]
plt.legend(loc=2, fontsize=16)
plt.title('Our activation functions', fontsize=20)
plt.ylim([-2, 5])
plt.xlim([-6, 6])
plt.show()

## Exercise c) Glorot initialization for all activation functions

Implement a function by adding to the code snippet below that can take network L and list of activations function as argument and return a Glorot initialized network.  Hint: [This blog post](https://mmuratarat.github.io/2019-02-25/xavier-glorot-he-weight-init) gives a table for the activation functions we use here.

Briefly explain in words how these how these values are calculated.

In [ ]:
def init_NN_Glorot(L, activations):
    """
    Initializer using the glorot initialization scheme
    """
    # Insert code here

# Initializes the unit test neural network
L_UT  = [3, 5, 1]
ACT_UT = [ReLU, Tanh]
NN_Glorot = init_NN_Glorot(L_UT, ACT_UT)

# Numpy einsum (EINstein SUMmation)

[Einsum](https://docs.scipy.org/doc/numpy/reference/generated/numpy.einsum.html) gives us the possibility to compute almost any matrix operation in a single function. You can find a good description in the link above. Here are a few examples of some important uses:

**Transpose:** We can write the transpose of matrix $A$:

```
np.einsum('ij -> ji', A) 
```

**Trace:** We can write the trace of matrix $A$:

```
np.einsum('ii -> ', A) 
```

**Diagonal:** We can write the diagonal of matrix $A$:

```
np.einsum('ii -> i', A) 
```
 
**Matrix product:** We can write the multiplication of matrices $A$ and $B$ as:

```
np.einsum('ij, jk -> ik', A, B)
```

Note that $j$ in both matrices $A$ and $B$ should be the same size. 

**Batched matrix product (or why bothering):** All of the functions we performed above are built in numpy (np.tranpose, np.trace, np.matmul), however, when you want to do more complex operations, it might become less readable and computationaly efficient. Let's introduce a three dimensional matrix $H$ with indices $b,j,k$, where the first dimension is the batch (training example) dimension. In einsum, we can then write:

```
np.einsum('ij, bjk -> bik', A, H)
```

In order to perform a batched matrix multiplication where we multiple over the second dimension in the first marix and second dimension in the second matrix. The result is a new three dimensional matrix where the first dimension is the first dimension from $H$ and second is the first dimension from $A$ and last dimension the last dimension from $H$. This is a very simple one line (and readable) way to do matrix operations that will be very useful for neural network code. 


#### _**Tips and tricks when using einsum**_

At the beginning, einsum might be a bit difficult to work with. The most important thing to do when using it is keeping track of the dimensions of your input and output matrices. An easy way to keep track of these dimensions is by using some sort of naming convention. Just like in the batched matrix product above we used $b$ to denote the batch dimension. In all the functions of this notebook, we leave some convention of names of indexes for the einsum in the explanation of the functions. We hope you find them useful!

There are some other useful resources to understand numpy.einsum:

* [Olexa Bilaniuk's great blogpost on einsum]( https://obilaniu6266h16.wordpress.com/2016/02/04/einstein-summation-in-numpy/ )
* [Stackoverflow answer to: Understanding NumPy's einsum]( https://stackoverflow.com/q/26089893/8899404 )
* [Jessica Stringham post on einsum]( https://jessicastringham.net/2018/01/01/einsum/ )
* [Slides of einstein summation from oxford]( http://www-astro.physics.ox.ac.uk/~sr/lectures/vectors/lecture10final.pdfc )

# Forward pass

The forward pass has been implemented for you. Please note how we have used einsum to perform the affine tranformation.

#### Indices convention.

* $i$: input - layer $l$ dimension.
* $o$: output - layer $l+1$ dimension.
* $b$: batch size dimension.

<u>Attention</u>! 

By convention we consider column vectors.
Depending on your implementation,
sometimes you will need to transpose the matrix/vector dimensions. 

#### Matrices, Sums and Indices

<u>Remember</u>!

When we compute a matrix-vector product, the inner indices need to match; if we have $W \in R^{K \times J}$ and $z \in R^{I}$, we can compute the matrix-vector product only if
   *  $J=I$, then $Wz$ or 
   *  $K=I$, then $W^Tz$.
   
We need to transpose the matrix in the second case. Why?. Hint: inner indices matching).
In general these two matrix-vector products are different. So pay attention to dimensions!

Similarly, when we sum a matrix and a vector over a dimension, we can sum only if the dimensions match. Given the summation:
    $$\sum_{i=1}^I w_{ki}~z_{i},$$
we can sum only if $J=I$.

Index Contraction: after summing over an index, the index is contracted and the output is no more a function of that index. This means that if we sum over $i$, the result will be an object without index $i$.

#### Unpacking the forward pass. 

For each layer we want to compute a transformation between the activated units $z$ and the layer parameters $(W, b)$. In particular such transformation is an affine one, of the form:
$$a^{(l)} = W^{(l)} z^{(l-1)} + b^{(l)},$$
followed by a non-linear function (activation)
$$z^{(l)} = h(a^{(l)}).$$

If $W_{oi}$ is an element in a matrix with $O$ rows and $I$ columns, 
$z_{i}$ is a value in a vector of $I$ units and 
$b_o$ is a value in a vector of $O$ biases, 
we can write the affine transformation in different ways:

* *Explicit notation*: $$\sum_{i=1}^{I} w_{oi}~z_{i} + b_o,   \quad \forall o=1,...O$$
* *Matrix notation*: $$ Wz + b$$ or $$z^{T}W + b$$


Given that we have only two indices ($i$ and $o$), after contracting $i$, the output will have dimension $O$.

#### Not so fast!

In Deep Learning we have another dimension: the batch size for the data $x$. 
This additional dimension is needed to pack together samples from the dataset and parallelize computations.

So we typically work with matrices $W_{io}$ (notice how the indices are switched now) and vectors $z_i$ in batches, that we write $X = \{x_{bi}\}^{B, I}_{i=0, b=0}$ or $Z = \{z_{bi}\}^{B, I}_{i=0, b=0}$.

Why do we use $W_{io}$ and not $W_{oi}$ as before? In principle we can use both. In practice, given that we want the batch size as first dimension, it is simpler to use $W_{io}$ to match the inner dimensions.
As we said before, depending on your specific implementation, you will need to transpose some matrices.

$x_{bi}$ means that we process in parallel a batch of $B$ samples (for example $B$=64 images, where each image is a sample) with dimensionality $I$ (for images this dimension is between $10^{3}$ - $10^{6}$). The batch size cannot be too large (Why?) and shouldn't be too small (Why?).

#### einsum

If you try to write in numpy a matrix-matrix product or a matrix-tensor product, you will notice that things get coumbersome fast. Additionally, your data could be a structured object with 3/4/5 dimensions ($x_{bijk}$). It's easy to get confused and make some mistakes, in particular when some of such dimensions have the same numerical value but different meaning (imagine a batch of 100 samples with dimension 100).

The **einsum** function is explicitly summing over the dimension of choice taking care of the details related to batching in an efficient way. There is a striking similarity between the einsum notation and the explicit notation we have seen before. 

In the simple example we are working on, using einsum, matmul, or summing over an index is basically the same. 
For more complex problems in computer vision, natural language processing, generative modeling, etc. einsum helps dealing with the details.

#### Putting it all together.

Ok! Here we are. Let's write again the explicit form for the batch case. As before, $W \in R^{I \times O}$ (or $W^{T} \in R^{O \times I}$) is a matrix, $z_{bi}$ is a matrix (or a collection of $b$ row vectors), and $b_o$ is a vector (in practice the bias vector will be broadcasted for all the samples in a batch).

* *Explicit notation*: 
$$\sum_{i=1}^I z_{bi}~w_{io} + b_{o}, \quad \forall o, \quad \forall b$$
Before, contracting $i$, we ended up with an object of dimension $o$. Now we end up with an object of dimensions $b~\times~o$.


* *Matrix notation*:
$$ Z W + b $$

In [ ]:
def forward_pass(x, NN, activations):
    """
    This function performs a forward pass. 
    It saves lists for both affine transforms of units (a) and activated units (z)
    Input:
    x: The input of the network             (np.array of shape: (batch_size, number_of_features))
    NN: The initialized neural network      (tuple of list of matrices)
    activations: the activations to be used (list of functions, same len as NN)

    Output:
    a: A list of affine transformations, that is, all x*w+b.
    z: A list of activated units (ALL activated units including input x and output y).
    
    Shapes for the einsum:
    b: batch size
    i: size of the input hidden layer (layer l)
    o: size of the output (layer l+1)
    """
    z = [x]
    a = []
    
    for l in range(len(NN[0])):
        
        # layer l parameters (W, bias)
        W = NN[0][l]
        bias = NN[1][l]
        
        # \sum_{i} z^{l}_{bi} W^{l}_{io} in explicit notation
        # z * W                          in matrix notation
        Wz = np.einsum('bi, io -> bo', z[l], W)
        
        # z * W + bias
        Wzb = Wz + bias
        
        a.append(Wzb)                  # The affine transform z*w+bias
        z.append(activations[l](a[l])) # The non-linearity    
    
    return a, z

# Forward pass unit test

Below is a piece of code that takes a very particular setting of the network and inputs and test whether it gives the expected results.

In [ ]:
ACT_F_UT = [Linear, Linear]
test_a, test_z = forward_pass(np.array([[1,1,1]]), NN_UT, ACT_F_UT) # input has shape (1, 3) 1 batch, 3 features

# Checking shapes consistency
assert np.all(test_z[0]==np.array([1,1,1])) # Are the input vector and the first units the same?
assert np.all(test_z[1]==test_a[0])         # Are the first affine transformations and hidden units the same?
assert np.all(test_z[2]==test_a[1])         # Are the output units and the affine transformations the same?

# Checking correctnes of values
# First layer, calculate np.sum(np.array([1,1,1])*np.array([1,1,1]))+1 = 4
assert np.all(test_z[1] == 4.)
# Second layer, calculate np.sum(np.array([4,4,4,4,4])*np.array([1,1,1,1,1]))+1 = 21
assert np.all(test_z[2] == 21.)

# Loss functions

In order to perform a backward pass we need to define a loss function and its derivative with respect to the output of the neural network $y$

In [ ]:
def squared_error(t, y, derivative=False):
    """
    Computes the squared error function and its derivative 
    Input:
    t:      target (expected output)          (np.array)
    y:      output from forward pass (np.array, must be the same shape as t)
    derivative: whether to return the derivative with respect to y or return the loss (boolean)
    """
    if np.shape(t)!=np.shape(y):
        print("t and y have different shapes")
    if derivative: # Return the derivative of the function
        return (y-t)
    else:
        return 0.5*(y-t)**2

## Exercise d) Implement cross entropy loss

Insert code below to implement cross-entropy loss for general dimensionality of $t$.

In [ ]:
def cross_entropy_loss(t, y, derivative=False):
    """
    Computes the cross entropy loss function and its derivative 
    Input:
    t:      target (expected output)          (np.array)
    y:      output from forward pass (np.array, must be the same shape as t)
    derivative: whether to return the derivative with respect to y or return the loss (boolean)
    """
    ## Insert code here